In [ ]:
import openmm

In [ ]:
import polychrom
from polychrom import starting_conformations
from polychrom import polymerutils
from polychrom import forces
from polychrom import forcekits
from polychrom.simulation import Simulation
from polychrom import contactmaps

In [ ]:
import nglutils.nglutils as ngu # from https://github.com/mirnylab/nglutils
import nglview as nv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import chromospyce

## Chromatin structure simulations

### 1. Initial conformation

In [ ]:
# Number of monomers in a single chain of polymer
N = 1000

In [ ]:
# Random walk creates very large conformations with low density of points. 
xyz = starting_conformations.create_random_walk(1, N)
view_simple = ngu.xyz2nglview(xyz)
view_simple

In [ ]:
# Display the structure in a chromospyce widget
chromospyce.Widget(xyz)

In [ ]:
# To initialize a compact structure, use self-avoiding random walk on cubic grid
xyz = polychrom.starting_conformations.grow_cubic(N=N, boxSize=50, method='standard')
print(xyz)
view_simple = ngu.xyz2nglview(xyz)
view_simple

In [ ]:
vc = {
    "color": "red",
}
chromospyce.Widget(xyz, vc)

### 2. Forces

In [ ]:
# Let's increase the number of monomers
N = 4000

In [ ]:
dens = 0.1
box = (N / dens) ** 0.33  # adjust for the polymer density
data = polychrom.starting_conformations.grow_cubic(N, int(box) - 2)  # create compact structure

a = Simulation(
        # platform="cuda",
        platform="cpu",
        integrator="variableLangevin",
        error_tol=0.01,
        GPU = "0",
        collision_rate=0.03,
        N = len(data),
        PBCbox=[box, box, box],
        precision="mixed")

a.set_data(data)  # load the data into the simulations object

a.add_force(
    forcekits.polymer_chains(
        a,
        chains=[(0, None, 0)],
        bond_force_func=forces.harmonic_bonds,
        bond_force_kwargs={
            'bondLength':1.0,
            'bondWiggleDistance':0.1,
          },

        angle_force_func=forces.angle_force,
        angle_force_kwargs={
            'k':1.5
        },

        nonbonded_force_func=forces.polynomial_repulsive,
        nonbonded_force_kwargs={
            'trunc':1.5,
            'radiusMult':1.05,
        },

        except_bonds=True,

    )
)
a.local_energy_minimization()

In [ ]:
view_simple = ngu.xyz2nglview(data)
view_simple

In [ ]:
data = a.get_data()
view_simple = ngu.xyz2nglview(data)
view_simple

In [ ]:
chromospyce.Widget(data)

In [ ]:
a.do_block(steps=1000)
data = a.get_data()
view_simple = ngu.xyz2nglview(data)
view_simple

In [ ]:
chromospyce.Widget(data)